In [3]:
from numpy.core.fromnumeric import shape
import librosa
import pathlib
import numpy as np
import os
# import pywt
import csv
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import normalize, to_categorical
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import tensorflow as tf
from tensorflow import keras,nn
from tensorflow.keras import layers


def model2d(input_shape, num_classes):

    model = keras.Sequential(name='model2d')

    #LFLB1
    model.add(layers.Conv2D(filters=64,kernel_size=3,strides=1,padding='same',input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=2, strides=2))
    
    #LFLB2
    model.add(layers.Conv2D(filters=64,kernel_size=3,strides=1, padding='same', ))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=4, strides=4))
    #model.add(layers.MaxPooling2D(pool_size=2, strides=2))

    #LFLB3
    model.add(layers.Conv2D(filters=128,kernel_size=3,strides=1,padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=4, strides=4))
    #model.add(layers.MaxPooling2D(pool_size=2, strides=2))


    
    #LFLB4
    model.add(layers.Conv2D(filters=128,kernel_size=3,strides=1,padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=4, strides=4))

    #model.add(layers.Reshape((-1, 128)))
    model.add(layers.TimeDistributed(layers.Flatten()))
    
    #LSTM
    #model.add(layers.LSTM(256))
    model.add(layers.Bidirectional(layers.LSTM(256)))
    
    #model.add(keras.layers.Dense(128,activation=nn.relu))
    #model.add(layers.Dense(128,activation=nn.relu))
    model.add(layers.Dense(units=num_classes, activation='softmax'))

    model.summary()

    opt = keras.optimizers.Adam(learning_rate=0.0006, decay=1e-6)

    model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['categorical_accuracy'])

    return model

# Clear TensorFlow session to release resources
tf.keras.backend.clear_session()

# Ensure memory growth is set
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
else:
    print("No GPU found. Training will proceed on the CPU.")

def train(train_data_x, train_data_y, emotion, emotionNumber, epochs=10):
    model = model2d(input_shape=(128, 257, 3), num_classes=emotionNumber)
    model.summary()
    
    batch_size = 5
    
    es = EarlyStopping(monitor='val_categorical_accuracy', mode='max', verbose=1, patience=20)
    
    mc = ModelCheckpoint(path+'/model/'+emotion+'_max_model.keras', monitor='val_categorical_accuracy', mode='max', verbose=1, save_best_only=True)
    
    history = model.fit(train_data_x, train_data_y, validation_data=(test_data_x, test_data_y), epochs=epochs, batch_size=batch_size, verbose=2, callbacks=[es, mc])
    
    # Save the train and test accuracy/loss at each epoch to a CSV file
    with open(path + '/model/' + emotion + '_training_history.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Epoch', 'Train Accuracy', 'Train Loss', 'Test Accuracy', 'Test Loss'])
        
        for epoch in range(epochs):
            train_acc = history.history['categorical_accuracy'][epoch] if epoch < len(history.history['categorical_accuracy']) else None
            train_loss = history.history['loss'][epoch] if epoch < len(history.history['loss']) else None
            val_acc = history.history['val_categorical_accuracy'][epoch] if epoch < len(history.history['val_categorical_accuracy']) else None
            val_loss = history.history['val_loss'][epoch] if epoch < len(history.history['val_loss']) else None
            
            writer.writerow([epoch + 1, train_acc, train_loss, val_acc, val_loss])
    
    acc = history.history['categorical_accuracy'][-1]
    
    model.save(path+'/model/'+emotion+'_max_model.keras')
    
    return acc

def test(test_data_x, test_data_y, emotion):
    
    new_model = load_model(path+'/model/'+emotion+'_max_model.keras')
    history=new_model.evaluate(test_data_x, test_data_y, batch_size=1)
    predict=new_model.predict(test_data_x)
    return history[1]

def maxIndex(data):
  max=data[0]
  index=0
  for i in range(1,len(data)):
    if(max<data[i]):
      max=data[i]
      index=i
  return index  

def test_emotion(test_data_x, test_data_y, total, emotion, path):
    model_path = os.path.join(path, 'model', emotion + '_max_model.keras')
    
    # Check if the file exists before loading
    if not os.path.isfile(model_path):
        raise ValueError(f"File not found: {model_path}. Please ensure the file is an accessible `.keras` zip file.")
    
    new_model = load_model(model_path)
    
    # Ensure the labels are one-hot encoded for testing
    test_data_y = to_categorical(test_data_y, num_classes=total)
    
    # Evaluate the model
    history = new_model.evaluate(test_data_x, test_data_y, batch_size=5)
    print(f"Test accuracy: {history[1]}")
    
    # Predict and generate confusion matrix
    predictions = new_model.predict(test_data_x, batch_size=5)
    confusion_matrix = np.zeros((total, total))
    
    for i in range(len(test_data_y)):
        actual_label = np.argmax(test_data_y[i])
        predicted_label = np.argmax(predictions[i])
        confusion_matrix[actual_label][predicted_label] += 1
    
    # Ensure the directory exists
    save_dir = os.path.join(path, 'data')
    os.makedirs(save_dir, exist_ok=True)
    
    # Save the confusion matrix to CSV
    confusion_csv_path = os.path.join(save_dir, emotion + '_confusion.csv')
    with open(confusion_csv_path, "w", newline='') as my_csv:
        csvWriter = csv.writer(my_csv, delimiter=',')
        csvWriter.writerows(confusion_matrix)
    
    print(f"Confusion matrix saved to: {confusion_csv_path}")

In [8]:

def load_array_from_npy(file_name):
    return np.load(file_name)

path = './data/SUBESCOxUIU/train/SUBESCO'

train_data_x = load_array_from_npy(path + '/npy/train_data_x.npy')
train_data_y = load_array_from_npy(path + '/npy/train_data_y.npy')

test_data_x = load_array_from_npy('./data/SUBESCOxUIU/test/SUBESCO/npy/test_data_x.npy')
test_data_y = load_array_from_npy('./data/SUBESCOxUIU/test/SUBESCO/npy/test_data_y.npy')

print(train_data_x.shape)
print(train_data_y.shape)
print(test_data_x.shape)
print(test_data_y.shape)

train_data_y = to_categorical(train_data_y)
test_data_y = to_categorical(test_data_y)

emotion='SUBESCO_latest'

acc=train(train_data_x, train_data_y, emotion, 5, 60)

(4000, 128, 257, 3)
(4000,)
(1000, 128, 257, 3)
(1000,)
Model: "model2d"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 128, 257, 64)      1792      
                                                                 
 batch_normalization_4 (Batc  (None, 128, 257, 64)     256       
 hNormalization)                                                 
                                                                 
 activation_4 (Activation)   (None, 128, 257, 64)      0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 64, 128, 64)      0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 64, 128, 64)       36928     
                                                                 
 ba

In [7]:
def test_emotion(test_data_x, test_data_y, total, emotion, path):
    model_path = os.path.join(path, 'model', 'SUBESCOxUIU_latest_max_model.keras')
    
    print(model_path)
    
    # Check if the file exists before loading
    if not os.path.isfile(model_path):
        ..................
        raise ValueError(f"File not found: {model_path}. Please ensure the file is an accessible `.keras` zip file.")
    
    new_model = load_model(model_path)
    
    # Ensure the labels are one-hot encoded for testing
    test_data_y = to_categorical(test_data_y, num_classes=total)
    
    # Evaluate the model
    history = new_model.evaluate(test_data_x, test_data_y, batch_size=5)
    print(f"Test accuracy: {history[1]}")
    
    # Predict and generate confusion matrix
    predictions = new_model.predict(test_data_x, batch_size=5)
    confusion_matrix = np.zeros((total, total))
    
    for i in range(len(test_data_y)):
        actual_label = np.argmax(test_data_y[i])
        predicted_label = np.argmax(predictions[i])
        confusion_matrix[actual_label][predicted_label] += 1
    
    # Ensure the directory exists
    save_dir = os.path.join(path, 'data')
    os.makedirs(save_dir, exist_ok=True)
    
    # Save the confusion matrix to CSV
    confusion_csv_path = os.path.join(save_dir, emotion + '_confusion.csv')
    with open(confusion_csv_path, "w", newline='') as my_csv:
        csvWriter = csv.writer(my_csv, delimiter=',')
        csvWriter.writerows(confusion_matrix)
    
    print(f"Confusion matrix saved to: {confusion_csv_path}")
    

path = './data/SUBESCOxUIU/train'

# test_data_x = load_array_from_npy(path + '/saved/test_data_x.npy')
# test_data_y = load_array_from_npy(path + '/saved/test_data_y.npy')
def load_array_from_npy(file_name):
    return np.load(file_name)


test_data_x = load_array_from_npy('./data/SUBESCOxUIU/test/npy/test_data_x.npy')
test_data_y = load_array_from_npy('./data/SUBESCOxUIU/test/npy/test_data_y.npy')

print(test_data_x.shape)
print(test_data_y.shape)
print(emotion)
test_emotion(test_data_x, test_data_y, 5, 'TR-combo-TST-uiu', path)

(1292, 128, 257, 3)
(1292,)
SUBESCOxUIU_latest
./data/SUBESCOxUIU/train\model\SUBESCOxUIU_latest_max_model.keras
259/259 [==============================] - 19s 8ms/step - loss: 0.8840 - categorical_accuracy: 0.8529
Test accuracy: 0.8529411554336548
259/259 [==============================] - 2s 5ms/step
Confusion matrix saved to: ./data/SUBESCOxUIU/train\data\TR-combo-TST-uiu_confusion.csv
